# Get SDO data with SSW/IDL

- https://sdo.oma.be/
- https://sdo.oma.be/vso_idl.html

## Search & download AIA data

```
; Search AIA 171 Angstrom data for the 6th december 2010 with a cadence of 3600 seconds (1 hour)
IDL> results = VSO_SEARCH('6-dec-2010', '7-dec-2010', inst='aia', wave='171', sample=3600)
Records Returned : JSOC : 25/25

; Sort the results by date and print the date
IDL> results = results(SORT(cat.time.start))
% Expression must be a structure in this context: CAT.
% Execution halted at: $MAIN$          
IDL> print, results.time.start
2010-12-06T11:00:00 2010-12-06T13:00:00 2010-12-06T12:00:00 2010-12-06T20:00:00 2010-12-06T08:00:00 2010-12-06T03:00:33 2010-12-07T00:00:00 2010-12-06T14:00:00 2010-12-06T10:00:00 2010-12-06T05:00:00 2010-12-06T22:00:00
2010-12-06T21:00:36 2010-12-06T06:00:00 2010-12-06T09:00:00 2010-12-06T23:00:00 2010-12-06T02:00:00 2010-12-06T19:00:00 2010-12-06T04:00:00 2010-12-06T18:00:00 2010-12-06T17:00:00 2010-12-06T16:00:00 2010-12-06T01:00:00
2010-12-06T15:00:00 2010-12-06T07:00:00 2010-12-06T00:00:00

; Download the first three files
IDL> status = VSO_GET(results[0:2], pixels=4096, filenames=files, /rice,  /use_network)
% VSO_GET: This will download 3 file(s)
1 : http://netdrms01.nispdc.nso.edu/cgi-bin/netdrms/drms_export.cgi?series=aia__lev1;compress=rice;record=171_1070708435-1070708435
% SOCK_GET_MAIN: 12337920 bytes of aia.lev1.171A_2010-12-06T11_00_00.34Z.image_lev1.fits copied in 4.04 seconds.
2 : http://netdrms01.nispdc.nso.edu/cgi-bin/netdrms/drms_export.cgi?series=aia__lev1;compress=rice;record=171_1070712035-1070712035
% SOCK_GET_MAIN: 12335040 bytes of aia.lev1.171A_2010-12-06T12_00_00.35Z.image_lev1.fits copied in 4.01 seconds.
3 : http://netdrms01.nispdc.nso.edu/cgi-bin/netdrms/drms_export.cgi?series=aia__lev1;compress=rice;record=171_1070715635-1070715635
% SOCK_GET_MAIN: 12335040 bytes of aia.lev1.171A_2010-12-06T13_00_00.34Z.image_lev1.fits copied in 4.20 seconds.
% VSO_GET: Downloading completed

; The local filenames will be in the files variable
print, files
D:\home\notebooks\Solarsoft\aia.lev1.171A_2010-12-06T11_00_00.34Z.image_lev1.fits D:\home\notebooks\Solarsoft\aia.lev1.171A_2010-12-06T12_00_00.35Z.image_lev1.fits
D:\home\notebooks\Solarsoft\aia.lev1.171A_2010-12-06T13_00_00.34Z.image_lev1.fits
```

## Read AIA image & plot it

```
; Read the image and header of the AIA fits file located at files[0] (see above)
; Because SDO files are rice compressed you need to use the specialised read_sdo procedure for this
IDL> READ_SDO, files[0], header, image
 ----------------------------------------------------
| Uncompressing to> C:\Users\usr\AppData\Local\Temp\ |
 ----------------------------------------------------
 ---------------------------------------------------------------------------------------------------------------------------------------------------
| D:\home\notebooks\Solarsoft\aia.lev1.171A_2010-12-06T11_00_00.34Z.image_lev1.fits -> C:\Users\usr\AppData\Local\Temp\AIA20101206_110000_0171.fits |
 ---------------------------------------------------------------------------------------------------------------------------------------------------

; Improve the contrast with aia_intscale and plot the resulting image
image = AIA_INTSCALE(image, exptime=header.EXPTIME, wavelnth=header.WAVELNTH, bytescale=1)
plot_image, image

; Load AIA standard colour table and plot the image using it
AIA_LCT, rr, gg, bb, wavelnth=header.WAVELNTH, /load
plot_image, image

; Save the image to PNG
png_file = header.T_OBS + ".png"
WRITE_PNG, png_file, image
```

![](test.png)

![](test2.png)

## Run AIA prep

> Perform image registration (rotation, translation, scaling) of Level 1 AIA images, and update the header information.

```
; Prep the AIA fits file located at files[0] (see above) and save it to directory outdir
AIA_PREP, files[0], [0], /do_write_fits, outdir='.', /verbose

; Prep the AIA fits file located at files[0] (see above) and save the resulting image to prepped_image
AIA_PREP, files[0], [0], prepped_header, prepped_image
```

![](test_prep.png)

![](test2_prep.png)

## Search & download HMI data

```
; Search HMI magnetogram data for the 6th december 2010 with a cadence of 3600 seconds (1 hour)
IDL> results = VSO_SEARCH('6-dec-2010', '7-dec-2010', inst='hmi', physobs='los_magnetic_field', sample=3600)
Records Returned : JSOC : 24/24
Records Returned : NSO : 3/3

; Download the first three files
IDL> status = VSO_GET(results[0:2], pixels=4096, filenames=files, /rice,  /use_network)
% VSO_GET: This will download 3 file(s)
1 : http://netdrms01.nispdc.nso.edu/cgi-bin/netdrms/drms_export.cgi?series=hmi__M_45s;compress=rice;record=12573122-12573122
% SOCK_GET_MAIN: 15747840 bytes of hmi.m_45s.2010.12.06_12_01_30_TAI.magnetogram.fits copied in 15.61 seconds.
2 : http://netdrms01.nispdc.nso.edu/cgi-bin/netdrms/drms_export.cgi?series=hmi__M_45s;compress=rice;record=12573602-12573602
% SOCK_GET_MAIN: 15822720 bytes of hmi.m_45s.2010.12.06_18_01_30_TAI.magnetogram.fits copied in 8.21 seconds.
3 : http://netdrms01.nispdc.nso.edu/cgi-bin/netdrms/drms_export.cgi?series=hmi__M_45s;compress=rice;record=12573682-12573682
% SOCK_GET_MAIN: 15667200 bytes of hmi.m_45s.2010.12.06_19_01_30_TAI.magnetogram.fits copied in 6.00 seconds.
% VSO_GET: Downloading completed

; Search HMI continuum data for the 6th december 2010 with a cadence of 3600 seconds (1 hour)
IDL> results = VSO_SEARCH('6-dec-2010', '7-dec-2010', inst='hmi', physobs='intensity', sample=3600)
Records Returned : JSOC : 24/24
```